In [2]:
import os 

In [3]:
os.chdir("\\textsummar\\Text_Summarizer")
%pwd

'c:\\textsummar\\Text_Summarizer'

In [4]:
#updata config.yaml 
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_file: Path 
    unzip_dir: Path

In [5]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_directories 

In [6]:
#create config manager
class ConfigManager:
    def  __init__(
         self,
        config_fp= CONFIG_PATH,
        params_fp =  PARAMS_PATH):

        self.config = read_yaml(config_fp)
        self.params = read_yaml(params_fp)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        DI_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL =  config.source_URL,
            local_file= config.local_file,
            unzip_dir= config.unzip_dir
        )
        return DI_config

In [7]:
#update components 
import os 
import urllib.request as request 
from text_summarizer.logging import logger 
import zipfile 

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config 
    
    def download_file(self):
        if not os.path.exists(self.config.local_file):
            fname, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_file
            )
            logger.info(f"{fname} download, with following info: \n {header}")
        else:
            logger.info(f"file already exists")
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_file , 'r') as zipref:
            zipref.extractall(unzip_path)

In [11]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e 

[2023-09-07 23:15:21,778:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-09-07 23:15:21,786:INFO:common:yaml file: params.yaml loaded successfully]
[2023-09-07 23:15:21,786:INFO:common:created directory at:artifacts]
[2023-09-07 23:15:21,786:INFO:common:created directory at:artifacts/data_ingestion]
[2023-09-07 23:15:35,311:INFO:2017815252:artifacts/data_ingestion/data.zip download, with following info: 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2E44:47C8:404886:491422:64FA0C2F
Accept-Ranges: bytes
Date: Thu, 07 Sep 2023 17:45:34 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4722-BOM
X-Cache: MISS
X-Cache-Hits: 0